# Preprocessing data

Preparing abstracts of papers for Natural Language Processing.

## Imports

In [2]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 853.9 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 2.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.9 MB/s eta 0:00:00


In [3]:
import string
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/fani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/fani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/fani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/fani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Sample abstracts

Taking the abstracts of three papers to test data cleaning. The examples come from these papers:
- Raichle ME. The brain's default mode network. Annu Rev Neurosci. 2015 Jul 8;38:433-47. doi: 10.1146/annurev-neuro-071013-014030. Epub 2015 May 4. PMID: 25938726.
- Thiebaut de Schotten M, Forkel SJ. The emergent properties of the connected brain. Science. 2022 Nov 4;378(6619):505-510. doi: 10.1126/science.abq2591. Epub 2022 Nov 3. PMID: 36378968.
- Alahmari A. Blood-Brain Barrier Overview: Structural and Functional Correlation. Neural Plast. 2021 Dec 6;2021:6564585. doi: 10.1155/2021/6564585. PMID: 34912450; PMCID: PMC8668349.

In [4]:
default = "The brain's default mode network consists of discrete, bilateral and symmetrical cortical areas, in the medial and lateral parietal, medial prefrontal, and medial and lateral temporal cortices of the human, nonhuman primate, cat, and rodent brains. Its discovery was an unexpected consequence of brain-imaging studies first performed with positron emission tomography in which various novel, attention-demanding, and non-self-referential tasks were compared with quiet repose either with eyes closed or with simple visual fixation. The default mode network consistently decreases its activity when compared with activity during these relaxed nontask states. The discovery of the default mode network reignited a longstanding interest in the significance of the brain's ongoing or intrinsic activity. Presently, studies of the brain's intrinsic activity, popularly referred to as resting-state studies, have come to play a major role in studies of the human brain in health and disease. The brain's default mode network plays a central role in this work."
connected = "There is more to brain connections than the mere transfer of signals between brain regions. Behavior and cognition emerge through cortical area interaction. This requires integration between local and distant areas orchestrated by densely connected networks. Brain connections determine the brain's functional organization. The imaging of connections in the living brain has provided an opportunity to identify the driving factors behind the neurobiology of cognition. Connectivity differences between species and among humans have furthered the understanding of brain evolution and of diverging cognitive profiles. Brain pathologies amplify this variability through disconnections and, consequently, the disintegration of cognitive functions. The prediction of long-term symptoms is now preferentially based on brain disconnections. This paradigm shift will reshape our brain maps and challenge current brain models. "
blood = "The blood-brain barrier (BBB) is a semipermeable and extremely selective system in the central nervous system of most vertebrates, that separates blood from the brain's extracellular fluid. It plays a vital role in regulating the transport of necessary materials for brain function, furthermore, protecting it from foreign substances in the blood that could damage it. In this review, we searched in Google Scholar, Pubmed, Web of Science, and Saudi Digital Library for the various cells and components that support the development and function of this barrier, as well as the different pathways to transport the various molecules between blood and the brain. We also discussed the aspects that lead to BBB dysfunction and its neuropathological consequences, with the identification of some of the most important biomarkers that might be used as a biomarker to predict the BBB disturbances. This comprehensive overview of BBB will pave the way for future studies to focus on developing more specific targeting systems in material delivery as a future approach that assists in combinatorial therapy or nanotherapy to destroy or modify this barrier in pathological conditions such as brain tumors and brain stem cell carcinomas. "

In [5]:
# List with abstracts
original = [default, connected, blood]

In [6]:
#Abstracts into dataframe
abstracts_df = pd.DataFrame(original, columns=['original'])

In [7]:
abstracts_df

,original
0,The brain's default mode network consists of d...
1,There is more to brain connections than the me...
2,The blood-brain barrier (BBB) is a semipermeab...


## Processing data

Processing the abstract of a paper.

### Basic cleaning
- turning text into lowercase
- removing numbers
- removing punctuation
- removing spaces in the beginning and the end

### Tokenization
Turning string into list of individual words.

### Removing stopwords
Keeping only useful words.

### Lemmatization
Simplifying word forms

In [15]:
 def preprocess(sentence):
     # Basic Cleaning
     cleaned = sentence.lower()
     cleaned = ''.join(char for char in cleaned if not char.isdigit())

     for punctuation in string.punctuation:
         cleaned = cleaned.replace(punctuation, '') 

     cleaned = cleaned.strip()

     # Tokenization
     tokenized = word_tokenize(cleaned)

     # Remove Stopwords
     stop_words = set(stopwords.words('english'))
     tokenized_no_stopwords = [word for word in tokenized if word not in stop_words]

     # Lemmatization
     lemmatized = [WordNetLemmatizer().lemmatize(word, pos="v") for word in tokenized_no_stopwords]
     lemmatized = [WordNetLemmatizer().lemmatize(word, pos="n") for word in lemmatized]

     cleaned_sentence = " ".join(lemmatized)
     return cleaned_sentence

Processing abstracts with list comprehension

In [16]:
abstracts_df['cleaned'] = abstracts_df["original"].apply(preprocess)

In [17]:
abstracts_df

,original,cleaned
0,The brain's default mode network consists of d...,brain default mode network consist discrete bi...
1,There is more to brain connections than the me...,brain connection mere transfer signal brain re...
2,The blood-brain barrier (BBB) is a semipermeab...,bloodbrain barrier bbb semipermeable extremely...
